In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import plotly.plotly as py
from plotly.graph_objs import *
import plotly.tools as tls
py.sign_in('glemaitre', 'se04g0bmi2')

import mpld3

import seaborn as sns; sns.set()
sns.palplot(sns.color_palette("muted"))

import numpy as np

from scipy.stats import norm,rayleigh,rice
from scipy.optimize import curve_fit
from scipy.stats.mstats import mode

from skimage import exposure

# Joblib library
### Module to performed parallel processing
from joblib import Parallel, delayed
# Multiprocessing library
import multiprocessing

## Load the data

Impor the mat containing the gray levels information

In [ ]:
from scipy.io import loadmat

matfiles = loadmat('../data/mat/all_voxels.mat');

data = np.asmatrix(matfiles['data'])
data = data.astype(float)
label = np.ravel(matfiles['label'])
patient_sizes = np.ravel(matfiles['patient_sizes'])

Load only the dce dataset

In [ ]:
data_dce = np.asmatrix(data[:, 3:-1])

Create a class Patient to facilitate data management.

The data are the DCE and this sequence is composed of multiple series. Thus, there is a need to compute information for each serie composing the DCE:

* the minimum intensity of the serie,
* the maximum intensity of the serie,
* the pdf of the serie,
* the mean of the serie,
* the std of the serie,
* the parameters of a Gaussian fitting on the distribution.

In [ ]:
class Patient(object):
    def __init__(self, data):
        self.data = data.copy()
        # Allocation of the minimum and maximum array for the series
        self.max_int_serie = []
        self.min_int_serie = []
        # Allocate the pdf and the bin_edges
        self.pdf_serie = []
        self.bin_edges_serie = []
        # Allocate the mean and std of the data
        self.mean_data_serie = []
        self.median_data_serie = []
        self.std_data_serie = []
        self.gaussian_params_serie = []
        self.idx_mean = []
        self.idx_median = []
        self.mode_data_serie = []
        for serie in range(data.shape[1]):
            # Find the minimum and maximum for the given serie
            self.max_int_serie.append(np.max(self.data[:, serie]))
            self.min_int_serie.append(np.min(self.data[:, serie]))
            # Compute the histogram
            pdf, bin_edges = np.histogram(self.data[:, serie],
                                                    bins = (self.max_int_serie[-1] - self.min_int_serie[-1]),
                                                    density=True)
            # Append the histogram
            self.pdf_serie.append(pdf)
            self.bin_edges_serie.append(bin_edges)
        
            self.mean_data_serie.append(np.mean(self.data[:, serie]))
            self.std_data_serie.append(np.std(self.data[:, serie]))
            self.median_data_serie.append(np.ravel(np.median(self.data[:, serie], axis=0)))
            #tmp_mode, tmp_idx = mode(self.data[:, serie], axis=None)
            #self.mode_data_serie.append(np.ravel(tmp_mode)[0])
            
            #print serie
            #print self.mode_data_serie
            
            # Find the index of the mean and the median for this serie
            ##### WRONG
            def argmean(x):
                x_mean = np.mean(np.ravel(x))
                return (np.abs(x-x_mean)).argmin()
            
            def argmedian(x):
                x_median = np.median(np.ravel(x))
                return (np.abs(x-x_median)).argmin()
            
            self.idx_mean.append([serie, argmean(self.data[:, serie])])
            self.idx_median.append([serie, argmedian(self.data[:, serie])])
       
            # Fit a gaussian distribution to get mean and std
            self.gaussian_params_serie.append(norm.fit(self.data[:, serie]))

In [ ]:
#Build an histogram for each patient
patient_list = [];
for pt in range(np.size(patient_sizes)):

    if (pt == 0):
        start_idx = 0
        end_idx = patient_sizes[pt] - 1
    else:
        start_idx = np.sum(patient_sizes[0 : pt])
        end_idx = np.sum(patient_sizes[0 : pt + 1]) - 1
        
    # Create the patient data
    patient_list.append(Patient(data_dce[start_idx : end_idx]))

## By patient analysis

We can make a by patient analysis in order to perform the normalisation. We will first start with some visualisation.

#### Definition of the patient to consider

In [ ]:
# Define the patient that we want to processed
pat_vis = 3

#### Define a function to build the heatmap recursively

In [ ]:
def build_heatmap(patient):
    # We need to shift the minimum value of the dataset to zero first
    ### Find the absolute minimum and maximum over the different series
    pt_min_intensity = min(patient.min_int_serie)
    pt_max_intensity = max(patient.max_int_serie)
    ### Define the maximum range of values necessary
    range_intensities = pt_max_intensity - pt_min_intensity

    # Allocate the heatmap
    heatmap = np.zeros((len(patient.pdf_serie), int(range_intensities)))

    # Build the heatmap
    ### For each serie
    for s in range(len(patient.pdf_serie)):
    
        # Define the size of the current pdf
        arr_sz = int(patient.max_int_serie[s] - patient.min_int_serie[s])
        # Compute the necessary offset from which the pdf has to be shifted
        offset = int(patient.min_int_serie[s] - pt_min_intensity)
        r = range(int(offset), arr_sz + int(offset))
        
        heatmap[s, range(offset, arr_sz + offset)] = patient.pdf_serie[s]
        
    # Return the heatmap
    return heatmap

#### Function to build graph from an image

In [ ]:
def build_graph(im, alpha):
    from scipy.sparse import coo_matrix
       
    graph = np.empty((im.size, im.size), dtype=float)
    
    for y in np.arange(graph.shape[0]):
        # Come back to the pixel index
        px_idx = np.unravel_index(y, im.shape)
        if (px_idx[0] >= (im.shape[0] - 1) or
            px_idx[1] >= (im.shape[1] - 1)    ):
            continue
        # Get the pixel value 
        edge_val = im[px_idx]
        # Assign the verteces
        # Find the position of the verteces inside the graph
        p_1 = np.ravel_multi_index((px_idx[0]+1, px_idx[1]), im.shape)
        p_2 = np.ravel_multi_index((px_idx[0], px_idx[1]+1), im.shape)
        # Assign the edge value
        graph[y, p_1] = alpha*edge_val
        graph[y, p_2] = (1.-alpha)*edge_val
        
    return coo_matrix(graph)

#### Function to find shortest-like path

In [ ]:
def short_path_depend(graph, image, start, end, short_path_type):

    # If we use the route_through_array from sklearn
    if(short_path_type == 'route-through-array'):
        from skimage.graph import route_through_array

        # Call the function from skimage
        indices, weight = route_through_array(image, start, end, geometric=True)
        path_list = np.array(indices)
        
    # Or we can use the scipy shortest_path
    elif(short_path_type == 'shortest-path'):
        # Define a function to go from px to vertices
        def px2v(px, im_shape):
            return np.ravel_multi_index(px, im_shape)

        # Define a function to go from vertices to px
        def v2px(v, im_shape):
            return np.unravel_index(v, im_shape)
        
        from scipy.sparse.csgraph import shortest_path

        # Compute the shortest path for the whole graph
        d, p = shortest_path(graph, return_predecessors=True)

        # Find the shape of the image
        im_sz = image.shape
        
        # Initiate the path
        path_list = [end]

        # Find the previous points in a loop
        while (end != start):
            # Convert coord from px to v
            s_v = px2v(start, im_sz)
            e_v = px2v(end, im_sz)
    
            # Find the predecessor
            pred_v = p[s_v, e_v]
    
            # Convert into pixel
            pred_px = v2px(pred_v, im_sz)
            path_list.append(pred_px)
    
            # Update the last point of the path
            end = pred_px
    else:
        raise ValueError('This method is not implemented!!!!')
        
    # Convert the list into array
    path_list = np.array(path_list)
        
    # Clean the path serie by serie to have a unique value   
    cleaned_path = []
    for t_serie in range(np.size(image, 0)):
        # Find all the intensities corresponding to this serie
        poi = path_list[np.nonzero(path_list[:, 0] == t_serie)]
    
        # Compute the median of the second column
        med_path = np.median(poi[:, 1])
        cleaned_path.append([t_serie, med_path])

    # Convert list to array
    cleaned_path = np.round(np.array(cleaned_path))
    
    return cleaned_path

#### Fonction to shift given an estimator

In [ ]:
def shift_estimator(patient, estimator):
    for s in range(len(patient_list[pat_vis].pdf_serie)):
        # Create the normalized data
        ### Correction using the estimator
        patient.data[:, s] = np.round(np.subtract(patient.data[:, s], (estimator[s])))
        # Recompute the pdf
        # Find the minimum and maximum for the given serie
        patient.max_int_serie[s] = np.max(patient.data[:, s])
        patient.min_int_serie[s] = np.min(patient.data[:, s])
        # Compute the histogram
        pdf, bin_edges = np.histogram(patient.data[:, s],
                                      bins = int(np.round((patient.max_int_serie[s] - patient.min_int_serie[s]))), 
                                      density=True)
        # Append the histogram
        patient.pdf_serie[s] = pdf
        patient.bin_edges_serie[s] = bin_edges

#### Visualise the data for the first time

In [ ]:
ax = sns.heatmap(build_heatmap(patient_list[pat_vis]), cmap="jet")

#### Make a test to find the shortest path with the two different methods

In [ ]:
from scipy.ndimage.filters import gaussian_filter1d
from skimage import img_as_float

# Correct the alignment of all the patient
for pat_vis in range(len(patient_list)):
    
    print 'Treating the patient #{}'.format(pat_vis)

    param_alpha = .9
    param_std = 50.
    param_exp = 25

    # Build the heatmap from the data
    heatmap = build_heatmap(patient_list[pat_vis])
    heatmap = gaussian_filter1d(heatmap, param_std, axis=1)
    print 'Heatmap built'
    # Build the conjugate map
    #heatmap = np.exp(param_exp * heatmap)
    image = 1. - (heatmap / np.max(heatmap))
    # Compute the exponential map to increase highly the difference
    image = img_as_float(np.exp(param_exp * image))
    #image = np.log10(1. + param_exp * image)
    print 'Image built'

    # Build the graph from the image
    graph = build_graph(image, param_alpha)
    print 'Graph built'

    # Compute the shortest path
    ### Find the starting and ending point of the map
    def median_serie(patient, serie):
        return np.median(patient.data[:, serie], axis=0)

    ### Find the offset
    offset = int(np.min(patient_list[pat_vis].min_int_serie))

    start = (0, 
             int(np.ravel(median_serie(patient_list[pat_vis], 0))[0] - offset))
    end = (np.size(heatmap, 0) - 1, 
           int(np.ravel(median_serie(patient_list[pat_vis], np.size(heatmap, 0) - 1))[0] - offset))
    print 'Starting and ending point computed: {} - {}'.format(start, end)

    # Call the algorithm to fing the shortest path
    path_idx = short_path_depend(graph, image, start, end, 'shortest-path')

    plt.figure()
    ax = sns.heatmap(image, cmap="jet")
    sns.plt.scatter(path_idx[:, 1], 39 - path_idx[:, 0], label='mean', c='green', alpha=0.9)
    plt.show()

    # Remove the offset to the data
    path_idx[:, 1] += offset

    shift_estimator(patient_list[pat_vis], path_idx[:, 1])

    # Build the heatmap from the data
    heatmap = build_heatmap(patient_list[pat_vis])
    heatmap = gaussian_filter1d(heatmap, param_std, axis=1)
    print 'Heatmap built'
    # Build the conjugate map
    #heatmap = np.exp(param_exp * heatmap)
    image = 1. - (heatmap / np.max(heatmap))
    # Compute the exponential map to increase highly the difference
    image = img_as_float(np.exp(param_exp * image))
    print 'Image built'

    # Build the graph from the image
    graph = build_graph(image, param_alpha)
    print 'Graph built'

    # Compute the shortest path
    ### Find the offset
    offset = int(np.min(patient_list[pat_vis].min_int_serie))

    start = (0, -offset)
    end = (np.size(heatmap, 0) - 1, -offset)
    print 'Starting and ending point computed: {} - {}'.format(start, end)

    # Call the algorithm to fing the shortest path
    path_idx = short_path_depend(graph, image, start, end, 'route-through-array')

    plt.figure()
    ax = sns.heatmap(image, cmap="jet")
    sns.plt.scatter(path_idx[:, 1], 39 - path_idx[:, 0], label='mean', c='green', alpha=0.9)
    plt.show()

    # Remove the offset to the data
    path_idx[:, 1] += offset

    shift_estimator(patient_list[pat_vis], path_idx[:, 1])

    # Plot the final results
    plt.figure()
    ax = sns.heatmap(build_heatmap(patient_list[pat_vis]), cmap="jet")
    plt.show()

#### Compute the Mean Squared Estimator (MSE)

In [ ]:


estim = []
vari = []
# Go in each serie of the DCE
for s in range(len(patient_list[pat_vis].pdf_serie)):
    # s is the pdf and we want to estmiate the variance considering 0 as the true mean
    # We will compute: sum ip_i ** 2
    ### We have to create the i - find the middle of the bin and the right number of value
    i_arr = (patient_list[pat_vis].bin_edges_serie[s][:-1] + 
             patient_list[pat_vis].bin_edges_serie[s][1:]    ) / 2.
    ### Compute the variance
    ipi_arr = (i_arr ** 2) * patient_list[pat_vis].pdf_serie[s]
    
    ### The estimator correspond to the sum normalise by the number of bins
    estim.append(np.sum(ipi_arr))# / float(len(i_arr)))
    vari.append(np.var(patient_list[pat_vis].data[:, s]))
    
    
plt.plot(estim, label='our estimate')
plt.plot(vari, label='variance')
plt.legend()

# Save the estimator
patient_list[pat_vis].mse_est = estim
patient_list[pat_vis].mse_var = vari    

# Let it on the side

In [ ]:
from skimage import img_as_float

param_alpha = .99
param_std = 3
param_exp = 20

# Build the heatmap from the data
heatmap = build_heatmap(patient_list[pat_vis])
print 'Heatmap built'
# Build the conjugate map
image = 1. - (heatmap / np.max(heatmap))
# Use a gaussian filter per row
# image = gaussian_filter1d(image, param_std, 0)
# Compute the exponential map to increase highly the difference
image = np.log10((1. + param_exp * image))
print 'Image built'

plt.figure()
ax = sns.heatmap(image, cmap="jet")
plt.show()

In [ ]:
# Define the index of the patient to visualise
pat_vis = 1

pat_heatmap = []
#for pat_vis in range(len(patient_list)):

# Allocate the heatmap
### Find the maximum and minimum from all series
pt_min_intensity = min(patient_list[pat_vis].min_int_serie)
pt_max_intensity = max(patient_list[pat_vis].max_int_serie)
### Allocate the heat map with the right indexing
heatmap_z_raw = np.zeros((len(patient_list[pat_vis].pdf_serie),
                                      int(pt_max_intensity)));

heatmap_y = []
for s in range(len(patient_list[pat_vis].pdf_serie)):
    str_pt = 'Serie ' + str(s) + ' '
    heatmap_y.append(str_pt)
    heatmap_z_raw[s, range(int(patient_list[pat_vis].min_int_serie[s]),
                           int(patient_list[pat_vis].max_int_serie[s]))] = patient_list[pat_vis].pdf_serie[s]

stat_median = np.ravel(patient_list[pat_vis].median_data_serie).astype(int)
stat_mean = np.ravel(patient_list[pat_vis].mean_data_serie).astype(int)
    
ax = sns.heatmap(heatmap_z_raw, cmap="jet")
sns.plt.scatter(np.ravel(patient_list[pat_vis].median_data_serie), np.arange(39, -1, -1), label='mean', c='green', alpha=0.9)
sns.plt.scatter(np.ravel(patient_list[pat_vis].mean_data_serie), np.arange(39, -1, -1), label='median', c='red', alpha=0.9)
#sns.plt.scatter(np.ravel(patient_list[pat_vis].mode_data_serie), np.arange(39, -1, -1), label='median', c='blue', alpha=0.9)


In [ ]:

from skimage import img_as_ubyte

# Define the starting and ending seeds
start = [0, stat_median[0]]
end = [39, stat_median[-1]]

# Compute the inverse image

from skimage.filters import gaussian_filter

image = 1. - (heatmap_z_raw / np.max(heatmap_z_raw))

from scipy.ndimage.filters import gaussian_filter1d
std_gauss = 50

im2 = []
for r in image:
    im2.append(gaussian_filter1d(r, std_gauss))
im2 = np.array(im2)
#image = im2

#for c in image.T:
#    c = gaussian_filter1d(c, std_gauss)
    
image = np.exp(15*image)

# Adaptive Equalization
# Contrast stretching
#p2, p98 = np.percentile(image, (2, 98))
#image = exposure.rescale_intensity(image, in_range=(p2, p98))
#image = np.log(exposure.equalize_hist(image))

plt.figure()
ax = sns.heatmap(image, cmap="jet")

# Find the shortest path
start = [0, stat_median[0]]
end = [39, stat_median[-1]]

start = [0, 235]
end = [39, 240]



sns.plt.scatter(indices[1], np.abs(indices[0]-39), label='mean', c='blue', alpha=0.9)

#plot_data = Data([Heatmap(z=heatmap_z_raw, y=heatmap_y, colorscale='Jet')])
#layout = Layout(margin=Margin(l=0, r=0, b=0, t=0))
#plot_data = Data(pat_heatmap)
# Define a figure
#fig = Figure(data=plot_data)
#py.iplot(fig, filename='heatmap-dce')

In [ ]:

    
graph = build_graph(image)

graph = coo_matrix(graph)

In [ ]:


plt.figure()
ax = sns.heatmap(heatmap_z_raw, cmap="jet")
path_list = np.array(path_list)
sns.plt.scatter(path_list[:, 1], 39 - path_list[:, 0], label='mean', c='red', alpha=0.9)
sns.plt.scatter(np.ravel(patient_list[pat_vis].median_data_serie), np.arange(39, -1, -1), label='mean', c='green', alpha=0.9)

#### Clean the path by taking the median for each row of interest

In [ ]:


plt.figure()
ax = sns.heatmap(heatmap_z_raw, cmap="jet")
path_list = np.array(path_list)
sns.plt.scatter(cleaned_path[:, 1], 39 - cleaned_path[:, 0], label='mean', c='red', alpha=0.9)
sns.plt.scatter(np.ravel(patient_list[pat_vis].median_data_serie), np.arange(39, -1, -1), label='mean', c='green', alpha=0.9)

In [ ]:
# Define the index of the patient to visualise
pat_vis = 1

pat_heatmap = []
#for pat_vis in range(len(patient_list)):

# Allocate the heatmap
### Find the maximum and minimum from all series
pt_min_intensity = min(patient_list[pat_vis].min_int_serie)
pt_max_intensity = max(patient_list[pat_vis].max_int_serie)
### Allocate the heat map with the right indexing
heatmap_z_raw = np.zeros((len(patient_list[pat_vis].pdf_serie),
                                      int(pt_max_intensity)));

heatmap_y = []
for s in range(len(patient_list[pat_vis].pdf_serie)):
    str_pt = 'Serie ' + str(s) + ' '
    heatmap_y.append(str_pt)
    heatmap_z_raw[s, range(int(patient_list[pat_vis].min_int_serie[s]),
                           int(patient_list[pat_vis].max_int_serie[s]))] = patient_list[pat_vis].pdf_serie[s]
        
plot_data = Data([Heatmap(z=heatmap_z_raw, y=heatmap_y, colorscale='Jet')])
#layout = Layout(margin=Margin(l=0, r=0, b=0, t=0))
#plot_data = Data(pat_heatmap)
# Define a figure
fig = Figure(data=plot_data)
py.iplot(fig, filename='heatmap-dce')

In [ ]:
serie_pdf = []
for ind, serie in enumerate(patient_list[pat_vis].pdf_serie):
    serie_pdf.append(np.squeeze(np.asarray(heatmap_z_raw[ind,:])))

### Correction using only the mean, median, or graph path estimator 

In [ ]:
for s in range(len(patient_list[pat_vis].pdf_serie)):
    # Create the normalized data
    ### Correction using the median
    #patient_list[pat_vis].data[:, s] = np.round(patient_list[pat_vis].data[:, s] - (patient_list[pat_vis].median_data_serie[s]))
    patient_list[pat_vis].data[:, s] = np.round(np.subtract(patient_list[pat_vis].data[:, s], (patient_list[pat_vis].graph_path[s])))
    # Recompute the pdf
    # Find the minimum and maximum for the given serie
    patient_list[pat_vis].max_int_serie[s] = np.max(patient_list[pat_vis].data[:, s])
    patient_list[pat_vis].min_int_serie[s] = np.min(patient_list[pat_vis].data[:, s])
    # Compute the histogram
    pdf, bin_edges = np.histogram(patient_list[pat_vis].data[:, s],
                                  bins = int(np.round((patient_list[pat_vis].max_int_serie[s] - patient_list[pat_vis].min_int_serie[s]))), 
                                  density=True)
    # Append the histogram
    patient_list[pat_vis].pdf_serie[s] = pdf
    patient_list[pat_vis].bin_edges_serie[s] = bin_edges
    
    #plt.figure()
    #width = 0.7 * (bin_edges[1] - bin_edges[0])
    #center = (bin_edges[:-1] + bin_edges[1:]) / 2
    #plt.bar(center, pdf, align='center', width=width)
    #plt.show()

We can show the data with mean, median, or graph path estimator shifted

In [ ]:
pat_heatmap = []
#for pat_vis in range(len(patient_list)):

# Allocate the heatmap
### Find the maximum and minimum from all series
pt_min_intensity = np.abs(min(patient_list[pat_vis].min_int_serie))
pt_max_intensity = np.abs(min(patient_list[pat_vis].min_int_serie)) + max(patient_list[pat_vis].max_int_serie)
### Allocate the heat map with the right indexing
heatmap_z_raw = np.zeros((len(patient_list[pat_vis].pdf_serie), int(pt_max_intensity)))

heatmap_y = []
for s in range(len(patient_list[pat_vis].pdf_serie)):
    str_pt = 'Serie ' + str(s) + ' '
    heatmap_y.append(str_pt)
    
    arr_sz = int(np.abs(patient_list[pat_vis].min_int_serie[s]) + patient_list[pat_vis].max_int_serie[s])
    offset = pt_min_intensity - np.abs(patient_list[pat_vis].min_int_serie[s])
    print pt_min_intensity
    r = range(int(offset), arr_sz+int(offset))
        
    heatmap_z_raw[s, r] = patient_list[pat_vis].pdf_serie[s]
        
plot_data = Data([Heatmap(z=heatmap_z_raw, y=heatmap_y, colorscale='Jet')])
#layout = Layout(margin=Margin(l=0, r=0, b=0, t=0))
#plot_data = Data(pat_heatmap)
# Define a figure
fig = Figure(data=plot_data)
py.iplot(fig, filename='heatmap-dce')

We can plot the variation of the std over time

In [ ]:
plt.figure()
plt.plot(patient_list[pat_vis].std_data_serie, 'o-')
plt.xlabel('Serie')
plt.ylabel('Standard deviation variations')
plt.show()

# Not useful for the moment

For a single patient we will start to fit a rician distrubution for each dce serie.

#### Define the function to fit

In [ ]:
from collections import namedtuple

# For the Rician
def myRice(x, factor, v, shift, std):
    return rice.pdf(x, v, shift, std) / factor

# Define the parameters for the Rician
riceParameters = namedtuple('riceParameters',
                            ['factor', 'v', 'shift', 'std'])

# For the Gaussian
def myGaussian(x, factor, mean, std):
    return norm.pdf(x, mean, std) / factor

# Define the parameters for the Gaussian
gaussianParameters = namedtuple('gaussianParameters',
                                ['factor', 'mean', 'std'])

#### Fit the distribution of one patient

In [ ]:
db_max_intensity = max(patient_list[pat_vis].max_int_serie)
db_min_intensity = min(patient_list[pat_vis].min_int_serie)

# Get the initial guess for the rician distribution
def get_rice_initial_parameters(patient, pdf, serie):
    
    # Get the mean from the patient information
    v = patient.mean_data_serie[serie] / db_max_intensity
    std = patient.std_data_serie[serie] / db_max_intensity
    factor = db_max_intensity
    
    # Compute the cumulative sum of the pdf
    cdf = np.cumsum(pdf)
    loc = float(np.argmax(cdf > .01)) / db_max_intensity
    #loc = np.argmax(cdf > .01)/db_max_intensity
    
    return riceParameters(factor, v, loc, std)

# For each serie get the initial parameter
rice_init = [get_rice_initial_parameters(patient_list[pat_vis], pdf, s) for s, pdf in enumerate(serie_pdf)]
rice_fitted_param = []
rice_fiting_error = []
for s in range(len(patient_list[pat_vis].pdf_serie)):
    popt, pcov = curve_fit(myRice,
                           np.linspace(0, 1., len(np.squeeze(np.asarray(heatmap_z_raw[s, :])))),
                           np.squeeze(np.asarray(heatmap_z_raw[s, :])),
                           p0=(rice_init[s].factor, rice_init[s].v,
                               rice_init[s].shift, rice_init[s].std),
                           maxfev=10000)
    rice_fitted_param.append(riceParameters(popt[0], popt[1], popt[2], popt[3],))
    rice_fiting_error.append(pcov)
    
# Get the initial guess for the rician distribution
def get_gaussian_initial_parameters(patient, pdf, serie):
    
    # Get the mean from the patient information
    mean = patient.mean_data_serie[serie] / db_max_intensity
    std = patient.std_data_serie[serie] / db_max_intensity
    factor = db_max_intensity
        
    return gaussianParameters(factor, mean, std)

# For each serie get the initial parameter
gaussian_init = [get_gaussian_initial_parameters(patient_list[pat_vis], pdf, s) for s, pdf in enumerate(serie_pdf)]
gaussian_fitted_param = []
gaussian_fiting_error = []
for s in range(len(patient_list[pat_vis].pdf_serie)):
    popt, pcov = curve_fit(myGaussian,
                           np.linspace(0, 1., len(np.squeeze(np.asarray(heatmap_z_raw[s, :])))),
                           np.squeeze(np.asarray(heatmap_z_raw[s, :])),
                           p0=(gaussian_init[s].factor, gaussian_init[s].mean, gaussian_init[s].std),
                           maxfev=10000)
    gaussian_fitted_param.append(gaussianParameters(popt[0], popt[1], popt[2],))
    gaussian_fiting_error.append(pcov)

In [ ]:
def output_msg(init, fitted):
    output_line = "{0:24s} ---> {1}:{2}\n"
    msg = "\n    initialization \t\t fitted\n"
    for param_name in riceParameters._fields:
        msg = msg+(output_line.format(
            param_name+': '+eval('"{0}".format(init.'+'{0:s}'.format(param_name)+')'),
            param_name,
            eval('"{0}".format(fitted.'+'{0:s}'.format(param_name)+')')))
    return msg

for s in range(len(patient_list[pat_vis].pdf_serie)):
       
    plt.figure()
    x = np.linspace(0, 1., int(db_max_intensity))
    plt.plot(x, np.squeeze(np.asarray(heatmap_z_raw[s, :])), label='data')
    
    plt.plot(x, myRice(x,
                       rice_fitted_param[s].factor,
                       rice_fitted_param[s].v,
                       rice_fitted_param[s].shift,
                       rice_fitted_param[s].std), label='rician')
    plt.plot(x, myGaussian(x,
                           gaussian_fitted_param[s].factor,
                           gaussian_fitted_param[s].mean,
                           gaussian_fitted_param[s].std), label='gaussian')
    plt.legend()
    plt.show()

#### Estimation of the mean for each PDF

In [ ]:
for s in range(len(patient_list[pat_vis].pdf_serie)):
    if np.isnan(rice.mean(rice_fitted_param[s].v,
                       rice_fitted_param[s].shift,
                       rice_fitted_param[s].std)):
        print 'Mean using Gaussian estimation: {}'.format(norm.mean(gaussian_fitted_param[s].mean,
                                                                    gaussian_fitted_param[s].std))
    else:
        print 'Mean using Rician estimatation: {}'.format(rice.mean(rice_fitted_param[s].v,
                                                                    rice_fitted_param[s].shift,
                                                                    rice_fitted_param[s].std))
        print 'Mean using Gaussian estimation: {}'.format(norm.mean(gaussian_fitted_param[s].mean,
                                                                    gaussian_fitted_param[s].std))
    print ''

#### Correct the mean offset

In [ ]:
for s in range(len(patient_list[pat_vis].pdf_serie)):
    # Estimate the mean as previously presented
    if np.isnan(rice.mean(rice_fitted_param[s].v,
                       rice_fitted_param[s].shift,
                       rice_fitted_param[s].std)):
        mean_corr = norm.mean(gaussian_fitted_param[s].mean,
                      gaussian_fitted_param[s].std)
    else:
        mean_corr = rice.mean(rice_fitted_param[s].v,
                              rice_fitted_param[s].shift,
                              rice_fitted_param[s].std)
    # Make the correction
    patient_list[pat_vis].data[:, s] = np.round(patient_list[pat_vis].data[:, s] - (mean_corr * db_max_intensity))
    # Recompute the pdf
    # Find the minimum and maximum for the given serie
    patient_list[pat_vis].max_int_serie[s] = np.max(patient_list[pat_vis].data[:, s])
    patient_list[pat_vis].min_int_serie[s] = np.min(patient_list[pat_vis].data[:, s])
    # Compute the histogram
    pdf, bin_edges = np.histogram(patient_list[pat_vis].data[:, s],
                                  bins = int(np.round((patient_list[pat_vis].max_int_serie[s] - patient_list[pat_vis].min_int_serie[s]))), 
                                  density=True)
    # Append the histogram
    patient_list[pat_vis].pdf_serie[s] = pdf
    patient_list[pat_vis].bin_edges_serie[s] = bin_edges

#### Recompute the heatmap to see the improvement

In [ ]:
pat_heatmap = []
#for pat_vis in range(len(patient_list)):

# Allocate the heatmap
### Find the maximum and minimum from all series
pt_min_intensity = np.abs(min(patient_list[pat_vis].min_int_serie))
pt_max_intensity = np.abs(min(patient_list[pat_vis].min_int_serie)) + max(patient_list[pat_vis].max_int_serie)
### Allocate the heat map with the right indexing
heatmap_z_raw = np.zeros((len(patient_list[pat_vis].pdf_serie), int(pt_max_intensity)))

heatmap_y = []
for s in range(len(patient_list[pat_vis].pdf_serie)):
    str_pt = 'Serie ' + str(s) + ' '
    heatmap_y.append(str_pt)
    
    arr_sz = int(np.abs(patient_list[pat_vis].min_int_serie[s]) + patient_list[pat_vis].max_int_serie[s])
    offset = pt_min_intensity - np.abs(patient_list[pat_vis].min_int_serie[s])
    r = range(int(offset), arr_sz+int(offset))
        
    heatmap_z_raw[s, r] = patient_list[pat_vis].pdf_serie[s]
        
plot_data = Data([Heatmap(z=heatmap_z_raw, y=heatmap_y, colorscale='Jet')])
#layout = Layout(margin=Margin(l=0, r=0, b=0, t=0))
#plot_data = Data(pat_heatmap)
# Define a figure
fig = Figure(data=plot_data)
py.iplot(fig, filename='heatmap-dce')

# Part to work with gradient descent